# 🧪 Curvality GW170817 Bayesian Analysis (Upload Inside Notebook)
This version lets you upload the zip file directly from Colab.

In [ ]:
# 📁 Upload ZIP interactively
from google.colab import files
uploaded = files.upload()

In [ ]:
# 📂 Unzip it
!unzip -o Curvality_GW170817_Colab_Files.zip

In [ ]:
# 📦 Install dependencies
!pip install numpy scipy matplotlib pymc h5py arviz

In [ ]:
# 📊 Load H1 and L1 strain from HDF5
import h5py
import numpy as np
import matplotlib.pyplot as plt

with h5py.File('H-H1_LOSC_CLN_4_V1-1187007040-2048.hdf5', 'r') as f:
    h1_strain = f['strain']['Strain'][:]
    h1_dt = f['strain']['Strain'].attrs['Xspacing']

with h5py.File('L-L1_LOSC_CLN_4_V1-1187007040-2048.hdf5', 'r') as f:
    l1_strain = f['strain']['Strain'][:]
    l1_dt = f['strain']['Strain'].attrs['Xspacing']

In [ ]:
# 🔊 Compute PSD
from scipy.signal import welch

fs = int(1 / h1_dt)
f_h1, psd_h1 = welch(h1_strain, fs, nperseg=fs)
f_l1, psd_l1 = welch(l1_strain, fs, nperseg=fs)

plt.loglog(f_h1, psd_h1, label='H1 PSD')
plt.loglog(f_l1, psd_l1, label='L1 PSD')
plt.xlabel('Frequency (Hz)')
plt.ylabel('PSD (strain²/Hz)')
plt.title('Power Spectral Density')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# 🧮 Load Curvality residuals
import pandas as pd

res_df = pd.read_csv("curvality_residuals.csv")
res_df.plot(x=res_df.columns[0], y=res_df.columns[1], title='Curvality Residual Δh(t)')

In [ ]:
# 🔁 PyMC model
import pymc as pm

with pm.Model() as model:
    alpha = pm.Normal('alpha', mu=1.0, sigma=0.2)
    beta = pm.Normal('beta', mu=1.0, sigma=0.2)
    gamma = pm.HalfNormal('gamma', sigma=0.1)
    delta = pm.HalfNormal('delta', sigma=0.1)
    S_c = pm.HalfNormal('S_c', sigma=0.1)
    R_c = pm.HalfNormal('R_c', sigma=0.01)

    epsilon = 1e-21
    likelihood = pm.Normal('obs', mu=0, sigma=epsilon, observed=res_df[res_df.columns[1]])

    trace = pm.sample(1000, tune=1000, cores=2, return_inferencedata=True)

In [ ]:
# 📈 Plot posterior distributions
import arviz as az
az.plot_trace(trace);